## Data preprocess

In [11]:
import numpy as np
import pandas as pd


def one_hot_encoding(arr):
    s = set()
    _sd = dict()
    for e in arr:
        s.add(e)

    sl = list(s)
    for i in range(len(sl)):
        encode = np.zeros(len(sl))
        encode[i] = 1
        _sd[sl[i]] = encode

    encoded_data = []
    for k in range(len(arr)):
        encoded_data.append(_sd[arr[k]])

    return np.array(encoded_data)


def data_preprocess(filepath):
    df = pd.read_excel(filepath)
    data = df.to_numpy()

    ip = data[:, 2]

    encoded_ip = one_hot_encoding(ip)

    # print(encoded_ip)

    data[:, 2] = encoded_ip.tolist()

    # print(data)
    expanded_data = []
    for row in data:
        expanded_data.append(np.concatenate([row[:2], row[2], row[3:]]).tolist())

    expanded_data = np.array(expanded_data)

    for i in range(len(expanded_data)):
        for j in range(len(expanded_data[i])):
            expanded_data[i][j] = float(expanded_data[i][j])

    return expanded_data[:290]


filename = "outputv2_5_wait_5s_3(300)"

# Data preprocessing
file_path = f"D:\\model_fit\\training\\training_data\\{filename}.xlsx"
dataset = data_preprocess(file_path)

print(dataset)

[[1.58961e+01 5.43987e+05 0.00000e+00 ... 1.90939e+01 1.86147e+01
  3.10231e+01]
 [5.35940e+00 3.67995e+05 0.00000e+00 ... 1.83406e+01 1.87500e+01
  3.04527e+01]
 [1.67825e+01 8.01369e+05 1.00000e+00 ... 3.82353e+01 3.86018e+01
  8.54700e+00]
 ...
 [2.02960e+00 1.84967e+05 1.00000e+00 ... 1.88854e+01 1.80258e+01
  3.03514e+01]
 [3.31653e+01 2.73870e+05 1.00000e+00 ... 3.87302e+01 3.89831e+01
  2.94671e+01]
 [2.17619e+01 9.61845e+05 1.00000e+00 ... 1.91111e+01 1.91223e+01
  2.96610e+01]]


## XGBOOST - 1

In [12]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X, y = dataset[:, 1:], dataset[:, 0]


train_size = int(len(X) * 0.7)

# 拆分数据集为训练集和测试集
# split dataset to train dataset and test dataset

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# time series
# X_train = X[:train_size]
# X_test = X[train_size:]
# y_train = y[:train_size]
# y_test = y[train_size:]

In [13]:
from xgboost import XGBRegressor

# Create XGBoost regression model
model = XGBRegressor(
    objective="reg:squarederror",   # Loss function to minimize: squared error for regression  损失函数为平方损失函数, 
    n_estimators=100,  # Number of boosting rounds (number of trees)  迭代次数，即基础学习器的数量
    learning_rate=0.1,  # Boosting learning rate (controls the step size)  学习率，控制每次更新的步长
    max_depth=5,  # Maximum depth of a tree  树的最大深度
    min_child_weight=1,  # Minimum sum of instance weight (hessian) needed in a child  叶子节点最小权重
    subsample=0.8,  # Subsample ratio of the training instance (randomly sampled)  每棵树随机采样的比例
    colsample_bytree=0.8,  # Subsample ratio of columns when constructing each tree  每棵树随机选择的特征比例
    alpha=0.01,  # L1 regularization term on weights  L1 正则化项参数
    reg_lambda=0.01,  # L2 regularization term on weights  L2 正则化项参数
)

# model fit
model.fit(X_train, y_train)

XGBRegressor(alpha=0.01, base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.8, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=1, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, ...)

In [14]:
from sklearn.metrics import mean_absolute_error

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
mea = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Squared Error: {mea}")


print(f"    real    |    pred   \n")
for i in range(len(y_test)):
    print(f"    {y_test[i]}     |     {y_pred[i]}      ")

Mean Squared Error: 82.94759487184533
Mean Squared Error: 6.300307321887182
    real    |    pred   

    3.024     |     5.423722743988037      
    13.5913     |     9.629517555236816      
    0.9716     |     8.985937118530273      
    8.7023     |     10.3786039352417      
    6.4232     |     9.986623764038086      
    7.2369     |     11.892337799072266      
    25.7131     |     13.266594886779785      
    3.213     |     4.814769268035889      
    55.6861     |     21.40213394165039      
    29.0796     |     19.007740020751953      
    8.452     |     5.993277072906494      
    0.3898     |     7.358521461486816      
    20.191     |     19.404451370239258      
    21.9265     |     8.772747039794922      
    3.3666     |     7.424693584442139      
    5.7826     |     16.276622772216797      
    16.7158     |     15.810887336730957      
    21.3984     |     6.180476188659668      
    10.1     |     8.313441276550293      
    0.9432     |     7.3856348991394

## XGBRegressor

In [15]:
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error


# 创建 XGBoost 回归模型
model = XGBRegressor(
    objective="reg:squarederror",
    n_estimators=100,
    learning_rate=0.1,
    tree_method="hist",
    device="cuda",
)

# Define the parameter grid for GridSearchCV
param_grid = {
    "max_depth": [3],  # Range of maximum depth of trees to explore  扩展最大深度的范围
    "learning_rate": [0.1],  # Range of learning rates to explore  扩展学习率的范围
    "n_estimators": [100],  # Range of number of trees (boosting rounds) to explore  扩展树的数量范围
    "gamma": [0],   # Range of gamma parameter to explore (controls tree split) 扩展 gamma 参数范围
    "subsample": [1],  # Range of subsample ratios to explore  扩展子样本比例范围
    "colsample_bytree": [0.8, 0.7],  # Range of column subsample ratios for each tree to explore  扩展列采样比例范围
    "reg_alpha": [0, 0.5],  # Range of L1 regularization parameters to explore  扩展 L1 正则化参数范围
    "reg_lambda": [0, 0.5],  # Range of L2 regularization parameters to explore  扩展 L2 正则化参数范围
}


grid_search = GridSearchCV(
    estimator=model, param_grid=param_grid, cv=5, scoring="neg_mean_squared_error"
)
grid_search.fit(X_train, y_train)

print("Best parameters found: ", grid_search.best_params_)

# 使用最佳参数重新训练模型
# Re-train the model using the best parameters
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train, verbose=True)


preds = best_model.predict(X_test)

rmse = mean_squared_error(y_test, preds, squared=False)
print(f"RMSE: {rmse}")

mae = mean_absolute_error(y_test, preds)
print(f"MAE: {mae}")



avg = np.mean(y_test)

print(f"AVG: {avg}")

print(f"    real    |    pred   \n")
a = 0
for i in range(len(y_test)):
    if (abs(y_test[i] - y_pred[i])) / y_test[i] <= 0.1:
        a += 1
    print(
        f"    {y_test[i]}     |     {y_pred[i]}      ",
        end=f"|   diff: {abs(y_test[i] - y_pred[i])}      |       {(abs(y_test[i] - y_pred[i])) / y_test[i]}\n",
    )

print(f"Accuracy in 0.9: {round(100 * a / len(y_test), 3)}%")

Best parameters found:  {'colsample_bytree': 0.8, 'gamma': 0, 'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 100, 'reg_alpha': 0.5, 'reg_lambda': 0.5, 'subsample': 1}
RMSE: 9.668572746734284
MAE: 6.480495737246809
AVG: 13.999570689655172
    real    |    pred   

    3.024     |     5.423722743988037      |   diff: 2.399722743988037      |       0.7935591084616525
    13.5913     |     9.629517555236816      |   diff: 3.961782444763184      |       0.2914940031316492
    0.9716     |     8.985937118530273      |   diff: 8.014337118530273      |       8.248597281319753
    8.7023     |     10.3786039352417      |   diff: 1.6763039352417      |       0.192627688684796
    6.4232     |     9.986623764038086      |   diff: 3.5634237640380864      |       0.5547739077154824
    7.2369     |     11.892337799072266      |   diff: 4.655437799072265      |       0.6432917131744621
    25.7131     |     13.266594886779785      |   diff: 12.446505113220216      |       0.4840530746281162
 

d:\model_fit\.venv\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
